<a href="https://colab.research.google.com/github/jeongrok/graphdata_analysis/blob/main/CNN_GraphMolecularData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

import torch
import torch.nn as nn

import sklearn as sk
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [ ]:
smiles_data = pd.read_csv('/content/drive/MyDrive/CSE6740/classification.txt', sep=" ")

In [ ]:
smiles_data.shape


maxima, minima = '', 'a'*1000
for x in smiles_data['molecule']:
  if len(x) < len(minima):
    minima = x
  if len(x) > len(maxima):
    maxima = x

print(minima)
print(maxima)
print(len(minima))
print(len(maxima))
boost_X, boost_y = smiles_data['molecule'],smiles_data['label']

NCCS
C1=CC2=[N+]3C1=C(c1ccccc1)C1=CC=C4C(c5cc[n+]([Pt-2]([n+]6ccc(C7=C8C=CC9=[N+]8[Zn-2]8%10[N+]%11=C(C=CC%11=C(c%11ccccc%11)C%11=CC=C7[NH+]%118)C(c7ccccc7)=C7C=CC(=C9c8ccccc8)[NH+]7%10)cc6)([n+]6ccc(C7=C8C=CC9=C(c%10ccccc%10)C%10=[N+]%11C(=C(c%12ccccc%12)C%12=CC=C%13C(c%14ccccc%14)=C%14C=CC7=[N+]%14[Zn-2]%11([NH+]%12%13)[NH+]98)C=C%10)cc6)[n+]6ccc(C7=C8C=CC9=C(c%10ccccc%10)C%10=[N+]%11C(=C(c%12ccccc%12)C%12=CC=C%13C(c%14ccccc%14)=C%14C=CC7=[N+]%14[Zn-2]%11([NH+]%12%13)[NH+]98)C=C%10)cc6)cc5)=C5C=CC6=[N+]5[Zn-2]3([NH+]14)[NH+]1C(=C2c2ccccc2)C=CC1=C6c1ccccc1.O=S(=O)([O-])C(F)(F)F
4
580


In [ ]:
SMILES_CHARS = [' ',
                '#', '%', '(', ')', '+', '-', '.', '/',
                '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                '=', '@',
                'A', 'B', 'C', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Z',
                '[', '\\', ']',
                'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'l', 'n', 'o', 'p', 'r',
                's', 't', 'u']

smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )

def smiles_encoder( smiles, maxlen=200):
    X = np.zeros( ( maxlen, len( SMILES_CHARS ) ) )
    Y = np.zeros(maxlen)
    for i, c in enumerate( smiles ):
        X[i, smi2index[c] ] = 1
        Y[i] = smi2index[c]
    return X, Y

def smiles_decoder( X ):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smi[ i ]
    return smi

In [ ]:
maxlen = 625
mol = smiles_data.iloc[:,0].to_numpy()
label = smiles_data.iloc[:,1].to_numpy()

mol_svm = np.zeros((mol.shape[0],maxlen))
mol_cnn = np.zeros((mol.shape[0],maxlen, 61))

for j in range(mol.shape[0]):
  X, Y = smiles_encoder(mol[j],maxlen)
  mol_cnn[j,:,:] = X
  mol_svm[j,:] = Y

svm_train = mol_svm[0:30000,:]
svm_test = mol_svm[30000:,:]

mol_train = mol_svm[0:30000,:]
mol_test = mol_svm[30000:,:]

label_train = label[0:30000]
label_test = label[30000:]

In [ ]:
print(sum(label) / len(label))
print(sum(label))

0.035479099064707004
1487


In [ ]:
# mol_cnn = np.reshape(mol_cnn, (41912,1,25,25))
mol_cnn.shape

(41912, 625, 61)

In [ ]:
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mol_cnn, label, test_size=0.2, random_state=42)

# converting numpy to pytorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
for batchx, batchy in train_loader:
  print(batchx.shape)
  print(batchy.shape)
  break

torch.Size([32, 625, 61])
torch.Size([32, 1])


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(61, 64, kernel_size=3, stride = 1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3,stride = 1, padding=1)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3,stride = 1, padding=1)
        self.conv4 = nn.Conv1d(256, 512, kernel_size=3,stride = 1, padding=1)
        self.fc1 = nn.Linear(512 * (625 // 16), 512)
        self.fc2 = nn.Linear(512, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.permute(0,2,1)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(-1, 512 * (625//16))
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Instantiate the model, define loss function, and optimizer
model = CNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
from sklearn.metrics import f1_score

for epoch in range(100):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    total_loss = 0
    all_preds_train, all_labels_train = [], []
    with torch.no_grad():
        correct = 0
        total = 0
        all_predictions = []
        all_labels = []
        for batch_x, batch_y in train_loader:
          output = model(batch_x)
          total_loss += loss.item()
        # Convert logits to probabilities using sigmoid activation
          probabilities = torch.sigmoid(output)
          predictions = (probabilities > 0.5).float()
          all_preds_train.extend(predictions.cpu().numpy())
          all_labels_train.extend(batch_y.cpu().numpy())

    # Calculate F1 score on training set
    train_f1 = f1_score(all_labels_train, all_preds_train)
    print(train_f1)
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = correct / total
    f1 = f1_score(all_labels, all_predictions)
    print(f'Epoch {epoch + 1}/{100}, Test Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}')

print("Training Complete")

0.06838345431501323
Epoch 1/100, Test Accuracy: 0.9642, F1 Score: 0.0000
0.06838345431501323
Epoch 2/100, Test Accuracy: 0.9642, F1 Score: 0.0000
0.06838345431501323
Epoch 3/100, Test Accuracy: 0.9642, F1 Score: 0.0000
0.06838345431501323
Epoch 4/100, Test Accuracy: 0.9642, F1 Score: 0.0000
0.06838345431501323
Epoch 5/100, Test Accuracy: 0.9642, F1 Score: 0.0000
0.06838345431501323
Epoch 6/100, Test Accuracy: 0.9653, F1 Score: 0.1803
0.06838345431501323
Epoch 7/100, Test Accuracy: 0.9652, F1 Score: 0.1512
0.06838345431501323
Epoch 8/100, Test Accuracy: 0.9650, F1 Score: 0.0872
0.06838345431501323
Epoch 9/100, Test Accuracy: 0.9645, F1 Score: 0.2624
0.06838345431501323
Epoch 10/100, Test Accuracy: 0.9656, F1 Score: 0.2131
0.06838345431501323
Epoch 11/100, Test Accuracy: 0.9655, F1 Score: 0.1525
0.06838345431501323
Epoch 12/100, Test Accuracy: 0.9656, F1 Score: 0.1579
0.06838345431501323
Epoch 13/100, Test Accuracy: 0.9660, F1 Score: 0.2480
0.06838345431501323
Epoch 14/100, Test Accuracy

In [ ]:
# Best results after hyper parameter tuning (lr 0.0005)

Epoch 1/100, Test Accuracy: 0.9642, F1 Score: 0.0000
Epoch 2/100, Test Accuracy: 0.9642, F1 Score: 0.0000
Epoch 3/100, Test Accuracy: 0.9645, F1 Score: 0.0132
Epoch 4/100, Test Accuracy: 0.9659, F1 Score: 0.1227
Epoch 5/100, Test Accuracy: 0.9650, F1 Score: 0.0518
Epoch 6/100, Test Accuracy: 0.9654, F1 Score: 0.0938
Epoch 7/100, Test Accuracy: 0.9653, F1 Score: 0.1046
Epoch 8/100, Test Accuracy: 0.9654, F1 Score: 0.2246
Epoch 9/100, Test Accuracy: 0.9665, F1 Score: 0.1994
Epoch 10/100, Test Accuracy: 0.9653, F1 Score: 0.2402
Epoch 11/100, Test Accuracy: 0.9652, F1 Score: 0.1512
Epoch 12/100, Test Accuracy: 0.9640, F1 Score: 0.2844
Epoch 13/100, Test Accuracy: 0.9611, F1 Score: 0.2788
Epoch 14/100, Test Accuracy: 0.9643, F1 Score: 0.2274
Epoch 15/100, Test Accuracy: 0.9645, F1 Score: 0.2280
Epoch 16/100, Test Accuracy: 0.9577, F1 Score: 0.2431
Epoch 17/100, Test Accuracy: 0.9654, F1 Score: 0.2246
Epoch 18/100, Test Accuracy: 0.9565, F1 Score: 0.2744
Epoch 19/100, Test Accuracy: 0.9645, 